# HW7- Brian Flores

## CH7 T-SQL for data analysis

## Q4

Write a query against the dbo.Orders table that returns a row for employee, a column for each order year, and the count of orders
for each employee and order year


**Tables involved:** NORTHWINDS database, Creating view That is in TSQLv4 (dbo.orders)

**Desired output:**

```
empid       cnt2014     cnt2015     cnt2016
----------- ----------- ----------- -----------
1           1           1           1
2           1           2           1
3           2           0           2

```
***

## **Proposition, Table, Columns and the Predicate.**

**Proposition:** 

We want a comprehensive report on the employees in a and how many orders they fufilled in the years 2014 2015 and 2016. In this case without window functions we would join on 4 tables. However we will use a pivot operator that treats row data as a colum. Here is the sample template

```
SELECT <non-pivoted column>, [pivoted column 1], [pivoted column 2], ...
FROM (
    -- subquery that provides the data
) AS source_table
PIVOT (
    <aggregate function>(<pivot column>)
    FOR <pivot column> IN ([value1], [value2], ...)
) AS pivot_table;

```

**Tables:** `[dbo].[orders]` 

**Columns:** empid, orderdate

**Predicate:** 
No real predicate is provided as the main part that is being affected is done thanks to the pivot operator that transposes your deserired rows into a column format. Once it does it can apply aggregate columns and then alias them to what you want them to be
    


In [ ]:
-- Q4 Prep for both databases
USE TSQLV4;


DROP TABLE IF EXISTS dbo.Orders;
GO

CREATE TABLE dbo.Orders
(
  orderid   INT        NOT NULL,
  orderdate DATE       NOT NULL,
  empid     INT        NOT NULL,
  custid    VARCHAR(5) NOT NULL,
  qty       INT        NOT NULL,
  CONSTRAINT PK_Orders PRIMARY KEY(orderid)
);

INSERT INTO dbo.Orders(orderid, orderdate, empid, custid, qty)
VALUES
  (30001, '20140802', 3, 'A', 10),
  (10001, '20141224', 2, 'A', 12),
  (10005, '20141224', 1, 'B', 20),
  (40001, '20150109', 2, 'A', 40),
  (10006, '20150118', 1, 'C', 14),
  (20001, '20150212', 2, 'B', 12),
  (40005, '20160212', 3, 'A', 10),
  (20002, '20160216', 1, 'C', 20),
  (30003, '20160418', 2, 'B', 15),
  (30004, '20140418', 3, 'C', 22),
  (30007, '20160907', 3, 'D', 30);

SELECT * FROM dbo.Orders;

-- TSQL V7
USE Northwinds2022TSQLV7;


DROP TABLE IF EXISTS dbo.Orders;
GO

CREATE TABLE dbo.Orders
(
  orderid   INT        NOT NULL,
  orderdate DATE       NOT NULL,
  empid     INT        NOT NULL,
  custid    VARCHAR(5) NOT NULL,
  qty       INT        NOT NULL,
  CONSTRAINT PK_Orders PRIMARY KEY(orderid)
);

INSERT INTO dbo.Orders(orderid, orderdate, empid, custid, qty)
VALUES
  (30001, '20140802', 3, 'A', 10),
  (10001, '20141224', 2, 'A', 12),
  (10005, '20141224', 1, 'B', 20),
  (40001, '20150109', 2, 'A', 40),
  (10006, '20150118', 1, 'C', 14),
  (20001, '20150212', 2, 'B', 12),
  (40005, '20160212', 3, 'A', 10),
  (20002, '20160216', 1, 'C', 20),
  (30003, '20160418', 2, 'B', 15),
  (30004, '20140418', 3, 'C', 22),
  (30007, '20160907', 3, 'D', 30);

SELECT * FROM dbo.Orders;

In [ ]:
-- Q4 
USE TSQLV4;

SELECT 
    empid, 
    [2014] AS cnt2014, 
    [2015] AS cnt2015, 
    [2016] AS cnt2016
FROM (
    SELECT 
        empid,
        YEAR(orderDate) AS orderYear
    FROM [dbo].[Orders]
) AS D
PIVOT (
    COUNT(orderYear)
    FOR orderYear IN ([2014], [2015], [2016])
) AS P;

USE Northwinds2022TSQLV7;

SELECT 
    empid,
    [2014] AS cnt2014,
    [2015] AS cnt2015,
    [2016] AS cnt2016
FROM (
    SELECT 
        empid,
        YEAR(orderdate) AS orderYear
    FROM [dbo].[Orders]
) AS D
PIVOT (
    COUNT(orderYear)
    FOR orderYear IN ([2014], [2015], [2016])
) AS P;


# Propositions

Translating queries to problems.

## Wundows Functions, Described
This section gives a flavor to what Window functions can do and their purpose

**WF 1**

Achieve a running aggregrate behavior on what employees have done. We want to know how much each employee has sold according to each order month. Keep a running total as you go by each new ordermonth.

**Tables Involved:**`Sales.EmpOrders`

**Constraints**: No subqueries

**Takeaways**: Assuming we have access to this data we use the following syntax to achieve the running aggregate technique
```
<aggregrate_function> OVER (
    [Possible partition on col,
    {Order by related_col }
    {Explicit Predicate}
    ]
)
```

In [ ]:
USE TSQLV4;

SELECT empid, ordermonth, val,
  SUM(val) OVER(PARTITION BY empid
                ORDER BY ordermonth
                ROWS BETWEEN UNBOUNDED PRECEDING
                         AND CURRENT ROW) AS runval
FROM Sales.EmpOrders;

USE Northwinds2022TSQLV7;

WITH [EmpOrders] AS 
(   
    SELECT 
        O.EmployeeId,
        CONVERT(date, DATEADD(MONTH, DATEDIFF(MONTH, 0, O.OrderDate), 0)) AS ordermonth,
        SUM(OD.Quantity * OD.UnitPrice) AS val
    FROM 
        [Sales].[Order] AS O
    INNER JOIN 
        [Sales].[OrderDetail] AS OD ON O.OrderId = OD.OrderId
    GROUP BY O.EmployeeId, CONVERT(date, DATEADD(MONTH, DATEDIFF(MONTH, 0, O.OrderDate), 0))
   
)
SELECT 
    EmployeeId,
    ordermonth,
    val,
    SUM(val) OVER (
                PARTITION BY EmployeeId
                ORDER BY ordermonth
                ROWS BETWEEN UNBOUNDED PRECEDING
                    AND CURRENT ROW) AS runval
FROM 
    [EmpOrders];




## Aggregrate Window Functions

**AWF 1**

show the total value of all transactions but also display thes sum value for each customer id

**Tables Involved:**`Sales.OrderValues`

**Constraints**: No Joins

**Takeaways**: 
The OVER() with no args exposes the window to all rows from the underlying rows.
The OVER(PARTITION BY custid) Restricts the window to work on by only focuses on each custid.

In [ ]:
-- AWF 1

-- Original TSQLV4
USE TSQLV4;

SELECT orderid, custid, val,
  SUM(val) OVER() AS totalvalue,
  SUM(val) OVER(PARTITION BY custid) AS custtotalvalue
FROM Sales.OrderValues;


-- Apadted Northwinds
USE Northwinds2022TSQLV7;

SELECT 
    orderid, 
    customerID, 
    val,
    SUM(val) OVER() AS totalvalue,
    SUM(val) OVER(PARTITION BY customerID) AS custtotalvalue
FROM Sales.OrderValues AS OV;


***
**AWF 2**

Show the fraction of a order id divided by total value generated across all orders. Also displau the percentage of that current orderid compared to sum of all the customer ids

**Tables Involved:**`Sales.OrderValues`

**Constraints**: No Joins

**Takeaways**: 

These window functions condenses sets of data into a scalar which can be achieved much like a subquery. The bottom query shows how this works as it simply does a scalar operation.

In [ ]:
-- AWF 2

-- Original TSQLv4
USE TSQLV4

SELECT orderid, custid, val,
  100. * val / SUM(val) OVER() AS pctall,
  100. * val / SUM(val) OVER(PARTITION BY custid) AS pctcust
FROM Sales.OrderValues;

-- Original Northwinds
USE Northwinds2022TSQLV7

SELECT 
    orderid, 
    customerID, 
    val,
    100. * val / SUM(val) OVER() AS totalvalue,
    100. * val / SUM(val) OVER(PARTITION BY customerID) AS custtotalvalue
FROM Sales.OrderValues AS OV;
